In [1]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from operator import itemgetter
from langchain_community.vectorstores import FAISS
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


In [2]:
model_type = 'llama'
name = 'llama3:70b'
model = Ollama(model=name)
embeddings = OllamaEmbeddings(model=name)

In [3]:
parser = StrOutputParser()

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [4]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

'My name is Santiago.'

In [5]:
pdf_path = "1Q23-EPR-with-Tables-FINAL.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()

In [6]:
vectorstore = FAISS.from_documents(pages, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [7]:
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [8]:
questions = [
    ("A company produces 500 tons of plastic packaging annually and is subject to an Extended Producer Responsibility (EPR) scheme "
     "with a recovery target of 80%. How many tons of plastic packaging should the company aim to recover each year to meet the EPR requirement?"),
     "What is the report quarter and when did it end?",
    ("What are the results for the reporting quarter? Please describe in the following order using bullet points - revenue, gross margin, opex, "
     "op margin, net income, and EPS. Include both GAAP and non-GAAP numbers. Please also include quarter-over-quarter changes."),
    ("If a municipality introduces a pay-as-you-throw program as part of its EPR strategy, charging households $0.05 per pound of collected waste, "
     "calculate the cost for a household that generates 30 pounds per week."),
    "What's the OP margin for the current quarter?",
    ("Identify potential risks associated with EPR implementation, such as regulatory changes, market volatility, and technological disruptions. "
     "Develop a risk assessment framework that assigns probabilities and impacts to each identified risk factor. Propose risk mitigation strategies "
     "for minimizing the adverse effects of these risks on EPR programs and stakeholder interests."),
     "What were Gap Inc's net sales for the first quarter of fiscal 2023 and how did they compare to the previous year?",
     "How did merchandise margin change from the previous year, and what were the drivers of this change?",
     "Did Gap Inc. have an operating profit or loss, and if so, what was the amount?",
     "What were the key results from each of Gap Inc.'s brands (Old Navy, Gap, Banana Republic, and Athleta)?",
     "What is the company's outlook on net sales for the second quarter and the full fiscal year 2023?",
     "How much cash and cash equivalents did Gap, Inc. have at the end of the quarter?",
     "How does the company expect gross margin to change in Q2 and the full year compared to 2022?",
     "Did Gap Inc. pay a dividend in the first quarter, and what is their dividend strategy going forward?",
     "What are the key actions Gap Inc. is taking to execute its plan?"
]

print(f"Total number of questions: {len(questions)}\n")
answers = []
for question in questions:
    ans = chain.invoke({'question': question})
    answers.append(ans)
    print(f"Question: {question}\n")
    print(f"Answer: {ans}")
    print('\n')

Total number of questions: 15

Question: A company produces 500 tons of plastic packaging annually and is subject to an Extended Producer Responsibility (EPR) scheme with a recovery target of 80%. How many tons of plastic packaging should the company aim to recover each year to meet the EPR requirement?

Answer: The company needs to recover 80% of its annual production of 500 tons. To calculate this, multiply 500 by 0.8:

500 x 0.8 = 400

So, the company should aim to recover 400 tons of plastic packaging each year to meet the EPR requirement.


Question: What is the report quarter and when did it end?

Answer: The report quarter is the first quarter of fiscal year 2023, and it ended on a date not specified in the provided text. However, based on the information that the sale of Gap China to Baozun Inc. closed on January 31, 2023, we can infer that the quarter likely ended around April 30, 2023.


Question: What are the results for the reporting quarter? Please describe in the followin

In [31]:
with open('answers.txt', 'w') as f:
    for ans in answers:
        f.write(f'{ans}\n')

## Evaluation

In [11]:
import evaluate

rouge = evaluate.load('rouge')

### Reference answers from ChatGPT.

The following answers were generated using chat gpt 4. We compute the ROGUE score with respect these answers.

In [33]:
ref_answers = [
    ("""To meet the Extended Producer Responsibility (EPR) requirement of an 80% recovery target, the company should aim to recover 80% of its annual plastic production. If the company produces 500 tons of plastic packaging annually, then:

\[ \text{Recovery Target} = 500 \, \text{tons} \times 80\% = 400 \, \text{tons} \]

Thus, the company should aim to recover 400 tons of plastic packaging each year to meet the EPR requirement."""),
("""The report is for the first quarter of fiscal year 2023, and it ended on April 29, 2023."""),
("""Here are the results for Gap Inc. for the first quarter of fiscal 2023, including both GAAP and non-GAAP metrics along with the quarter-over-quarter changes:

- **Revenue (Net Sales):**
  - **GAAP:** $3.276 billion, down 6% compared to last year's $3.477 billion.

- **Gross Margin:**
  - **GAAP:** 37.1%, an increase from the previous year's 31.5%.
  - **Non-GAAP:** 37.2%, slightly higher than the GAAP figure.

- **Operating Expenses (OpEx):**
  - **GAAP:** $1.224 billion, representing 37.4% of net sales.
  - **Non-GAAP:** $1.200 billion, representing 36.6% of net sales.

- **Operating Margin (Op Margin):**
  - **GAAP:** -0.3%, improved from the previous year's -5.7%.
  - **Non-GAAP:** 0.5%, reflecting adjustments for restructuring costs and a gain on sale of a building.

- **Net Income:**
  - **GAAP:** Net loss of $18 million, improved from the previous year's net loss of $162 million.
  - **Non-GAAP:** Net income of $3 million, adjusted for the gain on sale and restructuring costs.

- **Earnings Per Share (EPS):**
  - **GAAP:** Net loss per share of $0.05, improved from the previous year's net loss per share of $0.44.
  - **Non-GAAP:** Earnings per share of $0.01, adjusted for the same factors as net income.

These results reflect improvements in gross margin and operating margin due to reduced operating expenses and other cost management efforts, despite the overall decline in revenue."""),
("""To calculate the cost for a household that generates 30 pounds of waste per week under a pay-as-you-throw program where the charge is $0.05 per pound, you would multiply the amount of waste by the cost per pound:

\[ \text{Weekly Cost} = 30 \, \text{pounds} \times \$0.05/\text{pound} \]

\[ \text{Weekly Cost} = \$1.50 \]

Therefore, the cost for a household generating 30 pounds of waste per week would be $1.50."""),
("""The operating margin (Op Margin) for Gap Inc. in the first quarter of fiscal 2023, based on the GAAP figures, was -0.3%."""),
("""Implementing an Extended Producer Responsibility (EPR) program can encounter various risks stemming from regulatory, economic, technological, and operational challenges. Below is a proposed risk assessment framework that categorizes risks, estimates their likelihood and potential impact, and suggests mitigation strategies.

### 1. **Risk Identification**

#### A. Regulatory Changes
- **Description:** Changes in national or regional waste management policies and regulations could affect the operational parameters or cost structures of EPR programs.
- **Probability:** Moderate
- **Impact:** High

#### B. Market Volatility
- **Description:** Fluctuations in the recycling market, such as changes in the value of recovered materials, can impact the cost-effectiveness of EPR schemes.
- **Probability:** High
- **Impact:** Medium

#### C. Technological Disruptions
- **Description:** New technologies in product design or waste management (like biodegradable materials or enhanced recycling techniques) could render current EPR processes obsolete.
- **Probability:** Low
- **Impact:** High

#### D. Non-Compliance by Stakeholders
- **Description:** Producers, consumers, or intermediaries might fail to comply with their roles and responsibilities defined by the EPR framework.
- **Probability:** Moderate
- **Impact:** Medium

#### E. Public Acceptance and Behavior Changes
- **Description:** Resistance or slow adoption by the public to EPR initiatives, such as increased sorting efforts or acceptance of pay-as-you-throw schemes.
- **Probability:** Moderate
- **Impact:** Medium

### 2. **Risk Assessment Matrix**

Here’s a simple matrix to visualize the risks:

| Risk Factor                  | Probability | Impact | Priority |
|------------------------------|-------------|--------|----------|
| Regulatory Changes           | Moderate    | High   | High     |
| Market Volatility            | High        | Medium | Medium   |
| Technological Disruptions    | Low         | High   | Medium   |
| Non-Compliance by Stakeholders | Moderate  | Medium | Medium   |
| Public Acceptance            | Moderate    | Medium | Medium   |

### 3. **Risk Mitigation Strategies**

#### A. Regulatory Changes
- **Strategy:** Develop flexible EPR programs that can quickly adapt to regulatory changes. Engage in policy discussions and advocacy to influence favorable regulatory environments.
- **Tools:** Regulatory watch groups, lobbying efforts.

#### B. Market Volatility
- **Strategy:** Diversify recycling options and materials recovered. Develop financial reserves or insurance mechanisms to handle market downturns.
- **Tools:** Hedging strategies, partnerships with diverse markets.

#### C. Technological Disruptions
- **Strategy:** Invest in research and development. Stay informed about technological trends and integrate adaptable, forward-looking practices in the EPR framework.
- **Tools:** Innovation labs, collaborations with tech companies.

#### D. Non-Compliance by Stakeholders
- **Strategy:** Implement robust monitoring and enforcement mechanisms. Educate stakeholders about the benefits and obligations under EPR.
- **Tools:** Compliance audits, educational programs.

#### E. Public Acceptance and Behavior Changes
- **Strategy:** Conduct awareness campaigns, incentivize participation, and make EPR-related processes user-friendly.
- **Tools:** Public relations campaigns, user experience enhancements in recycling processes.

### 4. **Implementation and Review**

Regularly reviewing and updating the risk management framework is crucial as market conditions, regulatory environments, and technology evolve. Continuous feedback loops and stakeholder engagement will also help refine the approach and ensure the EPR program's resilience against these identified risks."""),
("""Gap Inc.'s net sales for the first quarter of fiscal 2023 were $3.276 billion. This represented a 6% decrease compared to the previous year, where the net sales were $3.477 billion."""),
("""In the first quarter of fiscal 2023, Gap Inc.'s merchandise margin increased by 600 basis points compared to the previous year. This improvement was primarily driven by lower air freight expenses and improved promotional activity during the quarter. These factors helped to offset inflationary cost headwinds, contributing to the substantial improvement in merchandise margin."""),
("""Gap Inc. reported an operating loss of $10 million for the first quarter of fiscal 2023, as per the GAAP metrics. This was an improvement from the previous year, where the operating loss was significantly higher at $197 million."""),
("""Here are the key results for each of Gap Inc.'s brands in the first quarter of fiscal 2023:

- **Old Navy:**
  - **Net Sales:** $1.8 billion, a decrease of 1% compared to the previous year.
  - **Comparable Sales:** Decreased by 1%.
  - **Performance Drivers:** The slight decline in sales was driven by continued softness in the active and kids' categories, as well as slower demand from lower-income consumers, offset by strength in the women’s category.

- **Gap Brand:**
  - **Net Sales:** $692 million, down 13% compared to last year.
  - **Comparable Sales:** Increased by 1%.
  - **Performance Drivers:** Sales were primarily driven by strength in the women’s category, offset by continued softness in the active and kids' categories as well as strategic store closures in North America. The decrease in net sales was further influenced by the negative impact from the sale of Gap China and foreign exchange headwinds.

- **Banana Republic:**
  - **Net Sales:** $432 million, a decrease of 10% on top of a 24% growth in the previous year.
  - **Comparable Sales:** Down 8%.
  - **Performance Drivers:** The sales decrease was impacted as the brand lapped significant growth from the previous year, which was driven by a shift in consumer preferences.

- **Athleta:**
  - **Net Sales:** $321 million, down 11% compared to last year.
  - **Comparable Sales:** Decreased by 13%.
  - **Performance Drivers:** Sales in the quarter were impacted by continued product acceptance challenges.

These results reflect varying challenges and strategic responses across the brands, ranging from market positioning shifts to adjustments in product offerings and strategic management of store operations."""),
("""Gap Inc. provided the following outlook for net sales:

- **Second Quarter of Fiscal 2023:** The company estimates that net sales could decrease in the mid to high-single digit range compared to last year’s net sales of $3.86 billion. This estimate accounts for the sale of Gap China to Baozun Inc., which closed on January 31, 2023. The net sales for the second quarter of the previous year included approximately $60 million from Gap China.

- **Full Fiscal Year 2023:** Gap Inc. anticipates that net sales for the entire fiscal year could decrease in the low to mid-single digit range compared to last year’s net sales of $15.6 billion. The outlook for fiscal 2023 also includes the impact of an additional 53rd week, which is estimated to positively impact net sales by $150 million. This is in addition to adjustments accounting for the sale of Gap China, which contributed approximately $300 million to the fiscal 2022 net sales.

These projections reflect ongoing uncertainties in the consumer and macroeconomic environment, as well as strategic adjustments made by the company in response to these conditions."""),
("""At the end of the first quarter of fiscal 2023, Gap Inc. had cash and cash equivalents totaling $1.2 billion, which represented an increase of 38% from the prior year."""),
("""Gap Inc. expects gross margin expansion for both the second quarter and the full fiscal year of 2023 compared to the prior year. This anticipated improvement reflects the company's ongoing efforts to manage costs effectively and optimize its operations, despite the uncertain consumer and macroeconomic environment. This positive outlook on gross margin indicates a focus on improving profitability across their brand portfolio."""),
("""Yes, Gap Inc. paid a dividend in the first quarter of fiscal 2023. The company paid a first quarter dividend of $0.15 per share, totaling $55 million. Additionally, the Board of Directors approved a second quarter fiscal 2023 dividend of $0.15 per share. This continuation of dividend payments indicates that despite the financial challenges, Gap Inc. remains committed to returning value to its shareholders through dividends, which is part of their broader financial strategy."""),
("""Gap Inc. has outlined several key actions to execute its strategic plan aimed at driving critical change and improving business performance. These actions include:

1. **Optimizing Operating Model:** Gap Inc. is focusing on simplifying its operating model and organizational structure. This includes reducing complexity and costs, which is intended to make the company more agile and efficient.

2. **Enhancing Promotional Strategy:** The company is working on improving its promotional activities to better manage inventory levels and increase merchandise margin, as evidenced by the significant improvement in this area in the first quarter.

3. **Investing in Digital and Omni-channel Initiatives:** Gap Inc. continues to invest in its digital capabilities and omni-channel shopping experiences to bridge the gap between online and physical store shopping, enhancing overall customer experience.

4. **Store Optimization:** The company is strategically managing its store footprint by closing underperforming locations and optimizing the performance of others, contributing to better overall financial health.

5. **Focusing on Core Brands:** Gap Inc. is prioritizing investments in its core brands—Old Navy, Gap, Banana Republic, and Athleta—to strengthen their market positions and cater to consumer preferences effectively.

6. **Leadership and Organizational Changes:** Gap Inc. has been engaging in leadership changes, including the search for a new CEO, to bring fresh vision and drive to the company’s strategic initiatives.

7. **Customer-Centric Approach:** A renewed focus on understanding and responding to customer needs and preferences is central to Gap Inc.'s strategy, aiming to improve brand loyalty and attract new customers.

These actions are part of Gap Inc.'s broader efforts to position itself for sustainable, profitable growth and to enhance shareholder value over the long term.""")
    
]

In [34]:
results = rouge.compute(predictions=answers, references=ref_answers)
print(results)

{'rouge1': 0.4927131407868678, 'rouge2': 0.2984815899656912, 'rougeL': 0.36922661186190137, 'rougeLsum': 0.42312921564222095}
